# *SixDman Basic Usage Example*

> This notebook demonstrates the basic usage of the SixDman package for optical metro-urban network planning.

In [30]:
# Import Dependencies
import sys
from pathlib import Path
import os
import numpy as np
import pandas as pd
import json

# Ignore warnings to have clean cell outputs
import warnings
warnings.filterwarnings("ignore")

### *Create results directory if it doesn't exist*

In [31]:
# Get the current working directory (where the notebook is running)
base_dir = Path.cwd()

# Define the results directory
results_dir = base_dir.parent / "results" / "single_level"

# Create it if it doesn't exist
results_dir.mkdir(exist_ok=True)

print(f"Results will be saved in: {results_dir}")

Results will be saved in: /media/matin/DataDisk/University/Master/Project/Codes/SixDman_GitHub_Repo/SixDman/results/single_level


In [32]:
sys.path.append(os.path.abspath(base_dir.parent / 'src'))
from sixdman.core.network import Network
from sixdman.core.band import Band, OpticalParameters
from sixdman.core.planning import PlanningTool
from sixdman.core.post_process import analyse_result

In [33]:
# Get the current working directory (where the notebook is running)
base_dir = Path.cwd()

# Define the results directory
results_dir = base_dir.parent / "results" / "single_level"

# Create it if it doesn't exist
results_dir.mkdir(exist_ok=True)

print(f"Results will be saved in: {results_dir}")

Results will be saved in: /media/matin/DataDisk/University/Master/Project/Codes/SixDman_GitHub_Repo/SixDman/results/single_level


### *1. Create Network Instance*

In [34]:
# Initialize network
network = Network(topology_name = 'MAN157')

# Load topology from .mat file
network.load_topology(filepath = '../data/MAN157Nodes.mat', matrixName ='MAN157Nodes')

# Set hierarchical levels
hl_dict = network.define_hierarchy(
    HL1_standalone = [1, 5],
    HL2_standalone = [0, 2, 3, 4],
    HL3_standalone = list(range(6, 39)),
    HL4_standalone = list(range(39, 157))
)

In [35]:
# Define variables for HL4 nodes
HL4_Standalone = hl_dict['HL4']['standalone']
HL4_colocated = hl_dict['HL4']['colocated']
HL4_all = np.concatenate((HL4_Standalone, HL4_colocated))

### *2. Calculate candidate paths for standalone and colocated nodes*

In [36]:
# Define subnet matrix of HL4 nodes
_, subnetMatrix_HL4 = network.compute_hierarchy_subgraph(hierarchy_level = 4, minimum_hierarchy_level = 4)

# Calculate connected nodes to standalone HL4 nodes (candidate destinations)
HL4_connected_nodes = network.get_neighbor_nodes(HL4_Standalone)

#### *2.1 Candidate paths for standalone HL4s*

In [37]:
# Define CSV file_name for candidate paths for standalone HL4 nodes
file_name = results_dir / f"{network.topology_name}_HL4_K_path_attributes.csv"

# Check if the CSV file exists
if file_name.exists():
    # If the file exists -> load the CSV file containing candidate paths for standalone HL4 nodes
    K_path_attributes_df = pd.read_csv(file_name)
    K_path_attributes_df['links'] = K_path_attributes_df['links'].map(json.loads)
    K_path_attributes_df['nodes'] = K_path_attributes_df['nodes'].map(json.loads)
    
else:   
    # If the file doesn't exist -> calculate candidate paths 

    # Define number of candidate paths per node
    k_paths = 20

    # define a list to store path attributes
    K_path_attributes = []

    # iterate through each standalone HL4 node
    for src in HL4_Standalone:
        for dest in HL4_connected_nodes:
            # Calculate k_paths candidate paths for (src, dest) pair
            K_path_attributes = network.compute_k_shortest_paths(subnetMatrix_HL4, K_path_attributes, source = src, target = dest, k = k_paths)

    # Convert K_path_attributes list to dataframe
    K_path_attributes_df = pd.DataFrame(K_path_attributes)

    # save dataframe to csv file
    K_path_attributes_df.to_csv(file_name, index = False)

#### *2.2 Candidate paths for colocated HL4s*

In [38]:
# Define CSV file_name for candidate paths for colocated HL4 nodes
file_name = results_dir / f"{network.topology_name}_HL4_K_path_attributes_colocated.csv"

# Check if the CSV file exists
if file_name.exists():
    # If the file exists -> load the CSV file containing candidate paths for colocated HL4 nodes
    K_path_attributes_colocated_df = pd.read_csv(file_name)
    K_path_attributes_colocated_df['links'] = K_path_attributes_colocated_df['links'].map(json.loads)
    K_path_attributes_colocated_df['nodes'] = K_path_attributes_colocated_df['nodes'].map(json.loads)
else:
    # If the file doesn't exist -> calculate candidate paths 
    
    # Define number of candidate paths per node
    k_paths = 20

    # define a list to store path attributes
    K_path_attributes_colocated = []

    # iterate through each colocated HL node
    for src in HL4_colocated:
        for dest in HL4_connected_nodes:
            if src != dest: # check that the src and dest node aren't same
                # Calculate k_paths candidate paths for (src, dest) pair
                K_path_attributes_colocated = network.compute_k_shortest_paths(subnetMatrix_HL4, K_path_attributes_colocated, source = src, target = dest, k = k_paths)

    # Convert K_path_attributes list to dataframe
    K_path_attributes_colocated_df = pd.DataFrame(K_path_attributes_colocated)

    # save dataframe to csv file
    K_path_attributes_colocated_df.to_csv(file_name, index = False)

#### *2.3 Sort dataframes based on num_hops and distance (in order)*

In [39]:
K_path_attributes_df_sorted = K_path_attributes_df.groupby(['src_node'], group_keys = False).apply(lambda x: x.sort_values(['num_hops', 'distance']))
K_path_attributes_colocated_df_sorted = K_path_attributes_colocated_df.groupby(['src_node', 'dest_node'], group_keys = False).apply(lambda x: x.sort_values(['num_hops', 'distance']))

In [40]:
# Check for the first five member of standalone candidate paths dataframe
K_path_attributes_df_sorted.head()

,src_node,dest_node,nodes,links,distance,num_hops
0,39,4,"[39, 40, 42, 4]","[118, 120, 27]",30.0,3
9,39,5,"[39, 41, 44, 45, 5]","[119, 121, 123, 31]",37.0,4
10,39,5,"[39, 41, 44, 43, 5]","[119, 121, 122, 30]",38.0,4
11,39,5,"[39, 40, 42, 4, 53, 52, 54, 5]","[118, 120, 27, 28, 131, 132, 33]",72.2,7
1,39,4,"[39, 41, 44, 45, 5, 54, 52, 53, 4]","[119, 121, 123, 31, 33, 132, 131, 28]",79.2,8


#### *2.4 Calculate LAND pairs for each standalone HL4*

In [41]:
pairs_disjoint = network.land_pair_finder(HL4_Standalone, K_path_attributes_df_sorted, num_pairs = 1)

### *3. Define Transmission Bands*

In [42]:
# Define C-band parameters
c_band_params = OpticalParameters()

# Create C-band instance
c_band = Band(
    name='C',
    start_freq = 190.65, # THz
    end_freq = 196.675, # THz
    opt_params = c_band_params,
    network_instance = network,
    channel_spacing = 0.05 # THz
    )

# Define L-band parameters
l_band_params = OpticalParameters()

# Create L-band instance
l_band = Band(
    name='L',
    start_freq = 184.525, # THz
    end_freq = 190.565, # THz
    opt_params = l_band_params,
    network_instance = network,
    channel_spacing = 0.05 # THz
)

In [43]:
# define C-band and L-band frequency slots
spectrum_C = c_band.calc_spectrum()
spectrum_L = l_band.calc_spectrum()

# concatenate C-band and KL-band to a sigle frequency spectrum
spectrum = np.concatenate((spectrum_C, spectrum_L))

# define total number of frequency slots
num_fslots = len(spectrum)

f_c_axis = spectrum * 1e12  # Convert to Hz
Pch_dBm = np.arange(-6, -0.9, 0.1)  # Channel power in dBm
num_Ch_mat = np.arange(1, len(spectrum) - 1)  # Channel indices

#### *3.1 Calculate HL4 links GSNR*

In [44]:
# Note that if the GSNR values calculated before, the function load the precalculated file
GSNR_opt_link, _, _, _ = c_band.process_link_gsnr(f_c_axis = f_c_axis, 
                                                  Pch_dBm = Pch_dBm, 
                                                  num_Ch_mat = num_Ch_mat,
                                                  spectrum_C = spectrum_C,
                                                  Nspan_array = np.ones(network.all_links.shape[0], dtype=int),
                                                  hierarchy_level = 4, 
                                                  minimum_hierarchy_level = 4,
                                                  result_directory = results_dir)

Loading precomputed link GSNR analysis


### *4. Create Planning Tool and Optimize Network*

In [45]:
# Initialize planning tool
planner = PlanningTool(
    network_instance = network,
    bands = [c_band, l_band], # simulate network over C and L Band
    period_time = 10) # simulate network for 10 years


#### *4.1 Simulating aggregated traffic at HL4s*

In [46]:
planner.initialize_planner(num_fslots = num_fslots,
                           hierarchy_level = 4, 
                           minimum_hierarchy_level = 4)

In [47]:
# generate port capacity for HL4 nodes uisng Monte Carlo simulation
# Note that if the initial traffic profile calculated before, the function load the precalculated profile
planner.generate_initial_traffic_profile(num_nodes = len(HL4_all),
                                monteCarlo_steps = 100,
                                min_rate = 20, # Gbps
                                max_rate = 200, # Gbps
                                seed = 20, result_directory = results_dir)

# Traffic growth simulation over 10 years
# Note that if the annual traffic profile calculated before, the function load the precalculated profile
planner.simulate_traffic_annual(lowest_hierarchy_dict = hl_dict['HL4'],
                                CAGR = 0.4, 
                                result_directory = results_dir)

Loading precomputed HL_capacity_final ...
Calculate Traffic Matrix ...


#### *4.2 Simulate network*

In [48]:
planner.run_planner(hierarchy_level = 4,
                    prev_hierarchy_level = 3,
                    pairs_disjoint = pairs_disjoint,
                    kpair_standalone = 1,
                    kpair_colocated = 1,
                    candidate_paths_standalone_df = K_path_attributes_df,
                    candidate_paths_colocated_df = K_path_attributes_colocated_df,
                    GSNR_opt_link = GSNR_opt_link,
                    minimum_level = 4, 
                    node_cap_update_idx = 2,
                    result_directory = results_dir)

# Note that the results will save in the results directory

Processing Year:  1
Processing Year:  2
Processing Year:  3
Processing Year:  4
Processing Year:  5
Processing Year:  6
Processing Year:  7
Processing Year:  8
Processing Year:  9
Processing Year:  10
